# TP4 - exercice 3 : glmnet & digits

Dans cet exercice nous allons illustrer l'utilisation de **glmnet** et des méthodes pénalisés parcimonieuses pour la classification mutliclasse.

Pour cela nous travaillerons sur le jeu de données **ZIP** que l'on peut télécharger [ici](https://web.stanford.edu/~hastie/ElemStatLearn/data.html). 
Comme les jeu **digits** et **MNIST**, ce jeu de données contient des imagettes de caractères manuscrits (des chiffres entre 0 et 1), mais avec une taille intermédiaire. Les images sont en effet de taille 16x16 (au lieu de 8x8 ou 28x28), ce qui réduit le temps nécessaire pour réaliser les analyses.

Le jeu de données se constitue d'un ensemble d'apprentissage et d'un ensemble de test, stockés dans les fichiers **zip.train** et **zip.test**. Chaque ligne de ces fichiers contient une instance et contient un chiffre entre 0 et 9 dans la 1ère colonne donnant la catégorie, puis les 256 valeurs définissant l'image.

## Question 1. Charger le jeu de données et représenter quelques images avec le code ci-dessous.
* NB: pour le TP on réduira même la taille du jeu d'apprentissage par 2 pour que l'exécution soit plus rapide.

In [ ]:
# generic imports #
#-----------------#
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# read training data #
#--------------------#
data_train = np.genfromtxt('datasets/zip-database/zip.train')
y_train = data_train[:,0]
X_train = data_train[:,1:]
# read test data #
#----------------#
data_test = np.genfromtxt('datasets/zip-database/zip.test')
y_test = data_test[:,0]
X_test = data_test[:,1:]
print('size of training data = %d x %d' % (X_train.shape[0],X_train.shape[1]))
print('size of test data = %d x %d' % (X_test.shape[0],X_test.shape[1]))
# show a few figures #
#--------------------#
# pick samples
ind_sple = np.random.choice(len(y_train), size = 8*8, replace = False)
# init plot
plt.figure(figsize=(6,6))
# plot images
plt_cptr=0
for i in range(8*8):
    plt_cptr = plt_cptr+1
    plt.subplot(8,8,plt_cptr)
    plt.imshow(np.reshape(X_train[ind_sple[i],:],(16,16)), cmap='binary')
    plt.xticks([])
    plt.yticks([])
plt.show()
      

In [ ]:
############################################
##### REDUCE SIZE OF TRAINING DATASET ######
############################################
ind_sple = np.random.choice(len(y_train), size = 3500, replace = False)
X_train = X_train[ind_sple,:]
y_train = y_train[ind_sple]
############################################

## Question 2. Reproduire l'expérience réalisée dans l'expercice 4 du TP2 visant à reconnaître le $0$ des autres chiffres par un modèle "Random Forest" avec le code ci-dessous.

In [ ]:
# define a 0-vs-other variable #
#------------------------------#
# for train
y_train_0 = np.zeros(y_train.shape)
y_train_0[y_train == 0] = 1
# for test
y_test_0 = np.zeros(y_test.shape)
y_test_0[y_test == 0] = 1

In [ ]:
# fit random forest model #
#-------------------------#
from sklearn.ensemble import  RandomForestClassifier
# instantiate model
rf = RandomForestClassifier(n_estimators = 500)
# learn model
rf.fit(X_train, y_train_0)

In [ ]:
# evaluate model #
#----------------#
y_pred_rf = rf.predict(X_test)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
# show variable importance #
#--------------------------#
var_imp = rf.feature_importances_
var_imp_mat = np.reshape(var_imp, (16,16))
plt.imshow(var_imp_mat, cmap ='binary')
plt.colorbar()
plt.show()

## Question 3. Effectuer la même analyse en considérant un modèle de régression logistique pénalisé par une norme $L_1$ (lasso), grâce au package *glmnet_python*.
* on optimisera le paramètre de régularisation grâce à la fonction **cvglmnet**.

In [ ]:
import glmnet_python

## Question 4. Considérer à présent le problème de classification multiclasse en utilisant un modèle multinomial. L'objectif principal étant de visualiser et comparer les modèles obtenus par différentes pénalités, on appliquera par souci de temps la procédure suivante : 
1. construire globalement le modèle (i.e., sans optimiser le paramêtre de régularisation) via la fonction **glmnet()** (en utilisant l'option $\texttt{family='multinomial'}$
2. évaluer ses performances **sur les jeux d'apprentissage et de test** via la fonction **glmnetPredict()**
3. représenter les performances obtenues le long du chemin de régularisation, **sur les jeux d'apprentissage et de test**.
4. représenter les modèles obtenus pour une "bonne" valeur de lambda : une valeur permettant d'obtenir de bonnes performances de test, tout en restant relativement parcimonieux. La ~40ème valeur de la grille obtenue par défaut devrait être un bon compromis.
### (bien entendu, la bonne manière de faire consisterait à optimiser le paramètre par validation croisée comme précédemment, et vous êtes tout à fait encouragés à réaliser cette analyse "pour de vrai" chez vous.)

## Les modèles obtenus sont-ils toujours interprétables ?

## Question 5. De la même manière, considérer le modèle multinomial pénalisé en norme $L_2$ (ridge) et représenter les modèles obtenus. Sont-ils plus interprétables ?
* là aussi, on se contentera de construire le modèle global, d'évaluer les performances train/test et de choisir une valeur de lambda
* la ~80ème valeur de la grille obtenue par défaut devrait être un bon compromis.

## Question 6. Réaliser la même analyse pour le modèle multinomial "groupé", i.e., pénalisé en group-lasso.
* là aussi, on se contentera de construire le modèle global, d'évaluer les performances train/test et de choisir une valeur de lambda
* la ~40ème valeur de la grille obtenue par défaut devrait être un bon compromis.

## Question 7. Enfin, visualiser les matrices de coefficients retenues, et compter le nombre total de variables retenues dans au moins un des modèles.
* on conservera les valeurs de lambda retenues précédemment.